In [1]:
#Q1

In [9]:
from ortools.sat.python import cp_model

size = 6
start = (1, 1)
target = (5, 5)
obstacles = [(2, 3), (3, 2), (4, 5)]
maxSteps = 10

model = cp_model.CpModel()

x = []
y = []

for t in range(maxSteps):
    xVar = model.NewIntVar(0, size - 1, f'x_{t}')
    yVar = model.NewIntVar(0, size - 1, f'y_{t}')
    x.append(xVar)
    y.append(yVar)

model.Add(x[0] == start[0])
model.Add(y[0] == start[1])

for t in range(maxSteps):
    for ox, oy in obstacles:
        model.AddForbiddenAssignments([x[t], y[t]], [(ox, oy)])

for t in range(maxSteps - 1):
    dx = model.NewIntVar(-1, 1, f'dx_{t}')
    dy = model.NewIntVar(-1, 1, f'dy_{t}')
    model.Add(dx == x[t+1] - x[t])
    model.Add(dy == y[t+1] - y[t])
    model.AddAllowedAssignments([dx], [[-1], [1]])
    model.AddAllowedAssignments([dy], [[-1], [1]])

reached = []
for t in range(maxSteps):
    flag = model.NewBoolVar(f'reached_{t}')
    xMatch = model.NewBoolVar(f'xMatch_{t}')
    yMatch = model.NewBoolVar(f'yMatch_{t}')

    model.Add(x[t] == target[0]).OnlyEnforceIf(xMatch)
    model.Add(x[t] != target[0]).OnlyEnforceIf(xMatch.Not())

    model.Add(y[t] == target[1]).OnlyEnforceIf(yMatch)
    model.Add(y[t] != target[1]).OnlyEnforceIf(yMatch.Not())

    model.AddBoolAnd([xMatch, yMatch]).OnlyEnforceIf(flag)
    model.AddBoolOr([xMatch.Not(), yMatch.Not()]).OnlyEnforceIf(flag.Not())

    reached.append(flag)


model.Add(sum(reached) >= 1)

model.Minimize(sum(reached[t] * t for t in range(maxSteps)))

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Path found:")
    for t in range(maxSteps):
        currentX = solver.Value(x[t])
        currentY = solver.Value(y[t])
        print(f"Step {t + 1}: ({currentX}, {currentY})")
        if (currentX, currentY) == target:
            break
else:
    print("No valid path")


Path found:
Step 1: (1, 1)
Step 2: (2, 2)
Step 3: (3, 3)
Step 4: (4, 4)
Step 5: (5, 5)


In [13]:
from ortools.sat.python import cp_model

Grid = [
    [0, 1, 1, 0, 0],
    [1, 1, 1, 0, 0],
    [1, 1, 0, 0, 1],
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
]

rows = len(Grid)
cols = len(Grid[0])
model = cp_model.CpModel()

boundary = [[model.NewBoolVar(f"flag{i}_{j}") for j in range(cols)] for i in range(rows)]

for i in range(rows):
    for j in range(cols):
        if Grid[i][j] == 0:
            model.Add(boundary[i][j] == 0)
        else:
            adjacent = []
            for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < rows and 0 <= nj < cols:
                    isWater = 1 if Grid[ni][nj] == 0 else 0
                    adjacent.append(isWater)
                else:
                    
                    adjacent.append(1)

            if sum(adjacent) > 0:
                model.Add(boundary[i][j] == 1)
            else:
                model.Add(boundary[i][j] == 0)

flags = [flag for row in boundary for flag in row]
perimeter = model.NewIntVar(0, rows * cols, "perimeter")
model.Add(perimeter == sum(flags))

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Perimeter:", solver.Value(perimeter))
    print("Map:")
    for i in range(rows):
        line = ""
        for j in range(cols):
            val = solver.Value(boundary[i][j])
            line += "B " if val else ". "
        print(line)
else:
    print("No solution found.")


Perimeter: 10
Map:
. B B . . 
B . B . . 
B B . . B 
. B . . . 
. B . B . 


In [15]:
from ortools.sat.python import cp_model

Matrix = [
    [0, 29, 20, 21, 16, 31, 100, 12, 4, 31],
    [29, 0, 15, 29, 28, 40, 72, 21, 29, 41],
    [20, 15, 0, 15, 14, 25, 81, 9, 23, 27],
    [21, 29, 15, 0, 4, 12, 92, 12, 25, 13],
    [16, 28, 14, 4, 0, 16, 94, 9, 20, 16],
    [31, 40, 25, 12, 16, 0, 95, 24, 36, 3],
    [100, 72, 81, 92, 94, 95, 0, 90, 101, 99],
    [12, 21, 9, 12, 9, 24, 90, 0, 15, 25],
    [4, 29, 23, 25, 20, 36, 101, 15, 0, 35],
    [31, 41, 27, 13, 16, 3, 99, 25, 35, 0]
]

cities = len(Matrix)
model = cp_model.CpModel()

nCity = [model.NewIntVar(0, cities - 1, f'nextCity_{i}') for i in range(cities)]

model.AddAllDifferent(nCity)

arcs = []
totalDistanceTerms = []

for i in range(cities):
    for j in range(cities):
        if i != j:
            isArc = model.NewBoolVar(f'arc_{i}_{j}')
            arcs.append([i, j, isArc])
            model.Add(nCity[i] == j).OnlyEnforceIf(isArc)
            model.Add(nCity[i] != j).OnlyEnforceIf(isArc.Not())

            dist = Matrix[i][j]
            totalDistanceTerms.append(isArc * dist)

model.AddCircuit(arcs)

totalDistance = model.NewIntVar(0, 10000, 'totalDistance')
model.Add(totalDistance == sum(totalDistanceTerms))
model.Minimize(totalDistance)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print("Tour found:")
    current = 0
    visited = set()
    for step in range(cities):
        print(f"Step {step + 1}: City {current}")
        visited.add(current)
        current = solver.Value(nCity[current])
    print(f"Return to start: City {current}")
    print("Total distance:", solver.Value(totalDistance))
else:
    print("No tour found.")

Tour found:
Step 1: City 0
Step 2: City 4
Step 3: City 3
Step 4: City 9
Step 5: City 5
Step 6: City 6
Step 7: City 1
Step 8: City 2
Step 9: City 7
Step 10: City 8
Return to start: City 0
Total distance: 246


In [1]:
from ortools.sat.python import cp_model

def CSP():
    model = cp_model.CpModel()

    Size = 6
    numRobots = 5
    numPackages = 10
    maxSteps = 15
    maxCapacity = 10
    maxBattery = 10

    weights = [2, 1, 3, 2, 2, 2, 3, 1, 2, 3]
    origins = [(0, 0), (1, 2), (2, 2), (3, 1), (4, 0),
               (5, 5), (0, 5), (1, 4), (2, 3), (3, 0)]
    destinations = [(5, 0), (4, 5), (0, 4), (1, 3), (3, 5),
                    (2, 0), (5, 2), (3, 3), (1, 1), (0, 2)]
    chargers = [(0, 0), (5, 5)]

    robotX, robotY, battery = {}, {}, {}
    for r in range(numRobots):
        for t in range(maxSteps):
            robotX[r, t] = model.NewIntVar(0, Size - 1, f"x_{r}_{t}")
            robotY[r, t] = model.NewIntVar(0, Size - 1, f"y_{r}_{t}")
            battery[r, t] = model.NewIntVar(0, maxBattery, f"battery_{r}_{t}")
        model.Add(battery[r, 0] == maxBattery)

    assignedRobot = {}
    for p in range(numPackages):
        assignedRobot[p] = model.NewIntVar(0, numRobots - 1, f"assigned_{p}")

    for r in range(numRobots):
        load = []
        for p in range(numPackages):
            isAssigned = model.NewBoolVar(f"robot{r}_hasPackage{p}")
            model.Add(assignedRobot[p] == r).OnlyEnforceIf(isAssigned)
            model.Add(assignedRobot[p] != r).OnlyEnforceIf(isAssigned.Not())
            usedWeight = model.NewIntVar(0, maxCapacity, f"weight_r{r}_p{p}")
            model.AddMultiplicationEquality(usedWeight, [isAssigned, weights[p]])  
            load.append(usedWeight)
        model.Add(sum(load) <= maxCapacity)

    for r in range(numRobots):
        for t in range(1, maxSteps):
            dx = model.NewIntVar(-1, 1, f"dx_{r}_{t}")
            dy = model.NewIntVar(-1, 1, f"dy_{r}_{t}")
            model.Add(dx == robotX[r, t] - robotX[r, t - 1])  
            model.Add(dy == robotY[r, t] - robotY[r, t - 1])  

            isMovingX = model.NewBoolVar(f"isMovingX_{r}_{t}")
            isMovingY = model.NewBoolVar(f"isMovingY_{r}_{t}")
            model.AddAbsEquality(model.NewIntVar(0, 1, ""), dx).OnlyEnforceIf(isMovingX) 
            model.AddAbsEquality(model.NewIntVar(0, 1, ""), dy).OnlyEnforceIf(isMovingY)  

            atCharger = model.NewBoolVar(f"atCharger_{r}_{t}")
            chargerConditions = []
            for cx, cy in chargers:
                isAtX = model.NewBoolVar("")
                isAtY = model.NewBoolVar("")
                model.Add(robotX[r, t] == cx).OnlyEnforceIf(isAtX)
                model.Add(robotX[r, t] != cx).OnlyEnforceIf(isAtX.Not())
                model.Add(robotY[r, t] == cy).OnlyEnforceIf(isAtY)
                model.Add(robotY[r, t] != cy).OnlyEnforceIf(isAtY.Not())
                match = model.NewBoolVar("")
                model.AddBoolAnd([isAtX, isAtY]).OnlyEnforceIf(match) 
                model.AddBoolOr([isAtX.Not(), isAtY.Not()]).OnlyEnforceIf(match.Not())  
                chargerConditions.append(match)
            model.AddMaxEquality(atCharger, chargerConditions) 

            drop = model.NewIntVar(0, 1, f"batDrop_{r}_{t}")
            model.Add(drop == 0).OnlyEnforceIf(atCharger)  
            model.Add(drop == 1).OnlyEnforceIf(atCharger.Not())  
            model.Add(battery[r, t] == battery[r, t - 1] - drop)

    for t in range(maxSteps):
        for r1 in range(numRobots):
            for r2 in range(r1 + 1, numRobots):
                sameX = model.NewBoolVar("")
                sameY = model.NewBoolVar("")
                model.Add(robotX[r1, t] == robotX[r2, t]).OnlyEnforceIf(sameX)
                model.Add(robotX[r1, t] != robotX[r2, t]).OnlyEnforceIf(sameX.Not())
                model.Add(robotY[r1, t] == robotY[r2, t]).OnlyEnforceIf(sameY) 
                model.Add(robotY[r1, t] != robotY[r2, t]).OnlyEnforceIf(sameY.Not())
                model.AddBoolOr([sameX.Not(), sameY.Not()]) 

    totalBatteryUsed = model.NewIntVar(0, numRobots * maxBattery, "totalBatteryUsed")
    batteryLeft = [battery[r, maxSteps - 1] for r in range(numRobots)]
    model.Add(totalBatteryUsed == numRobots * maxBattery - sum(batteryLeft))  
    model.Minimize(totalBatteryUsed)

    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 20.0
    status = solver.Solve(model)

    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for r in range(numRobots):
            print(f"\nRobot {r} path:")
            for t in range(maxSteps):
                posX = solver.Value(robotX[r, t])
                posY = solver.Value(robotY[r, t])
                bat = solver.Value(battery[r, t])
                print(f"  Time {t}: ({posX}, {posY}), Battery: {bat}")
        print("\nPackage Assignments:")
        for p in range(numPackages):
            print(f"  Package {p} → Robot {solver.Value(assignedRobot[p])}")
        print(f"\nTotal Battery Used: {solver.Value(totalBatteryUsed)}")
    else:
        print("No feasible solution found.")

CSP()

No feasible solution found.


In [15]:
from ortools.sat.python import cp_model

def solve(grid):
    model = cp_model.CpModel()

    size = 9
    subgrid = 3
    primes = [2, 3, 5, 7]

    cells = []
    for r in range(size):
        row = []
        for c in range(size):
            row.append(model.NewIntVar(1, 9, f'cell_{r}_{c}'))
        cells.append(row)

    for r in range(size):
        for c in range(size):
            if grid[r][c] != 0:
                model.Add(cells[r][c] == grid[r][c])

    for r in range(size):
        model.AddAllDifferent(cells[r])

    for c in range(size):
        col = []
        for r in range(size):
            col.append(cells[r][c])
        model.AddAllDifferent(col)

    for br in range(0, size, subgrid):
        for bc in range(0, size, subgrid):
            block = []
            for i in range(subgrid):
                for j in range(subgrid):
                    block.append(cells[br + i][bc + j])
            model.AddAllDifferent(block)

    main_diag = []
    anti_diag = []
    for i in range(size):
        main_diag.append(cells[i][i])
        anti_diag.append(cells[i][size - 1 - i])

    diag1_sum = model.NewIntVar(0, 81, "diag1_sum")
    diag2_sum = model.NewIntVar(0, 81, "diag2_sum")

    model.Add(diag1_sum == sum(main_diag))
    model.Add(diag2_sum == sum(anti_diag))

    model.AddModuloEquality(0, diag1_sum, 3)
    model.AddModuloEquality(0, diag2_sum, 3)

    for r in range(size):
        for c in range(size):
            for dr, dc in [(0, 1), (1, 0)]:
                nr = r + dr
                nc = c + dc
                if nr < size and nc < size:
                    for prime in primes:
                        cond1 = model.NewBoolVar("")
                        cond2 = model.NewBoolVar("")
                        model.Add(cells[r][c] == prime).OnlyEnforceIf(cond1)
                        model.Add(cells[r][c] != prime).OnlyEnforceIf(cond1.Not())
                        model.Add(cells[nr][nc] == prime).OnlyEnforceIf(cond2)
                        model.Add(cells[nr][nc] != prime).OnlyEnforceIf(cond2.Not())
                        both_prime = model.NewBoolVar("")
                        model.AddBoolAnd([cond1, cond2]).OnlyEnforceIf(both_prime)
                        model.AddBoolOr([cond1.Not(), cond2.Not()]).OnlyEnforceIf(both_prime.Not())

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        for r in range(size):
            row = []
            for c in range(size):
                row.append(solver.Value(cells[r][c]))
            print(row)
    else:
        print("No solution found.")

sudokuGrid = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0]
]


solve(sudokuGrid)


[7, 3, 9, 4, 5, 1, 8, 6, 2]
[8, 1, 4, 6, 2, 7, 5, 3, 9]
[6, 5, 2, 3, 8, 9, 1, 7, 4]
[2, 7, 3, 1, 9, 5, 4, 8, 6]
[9, 6, 5, 7, 4, 8, 3, 2, 1]
[1, 4, 8, 2, 6, 3, 9, 5, 7]
[3, 9, 1, 8, 7, 6, 2, 4, 5]
[5, 2, 6, 9, 3, 4, 7, 1, 8]
[4, 8, 7, 5, 1, 2, 6, 9, 3]
